In [309]:
import pandas as pd 
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime
from sqlalchemy.dialects.sqlite import DATETIME

In [310]:
df = pd.read_csv('../Resources/collision_2022.csv')

In [311]:
df = df.drop_duplicates(subset=['collision_id'], keep='first').set_index('collision_id')

In [312]:
df['zip_code'] = df['zip_code'].astype(str)

In [313]:
df['crash_datetime'] = pd.to_datetime(df['crash_datetime'])

In [314]:
df.drop(columns='location', inplace=True)

In [315]:
df.rename(columns={'vehicle_type_code1': 'vehicle_type_code_1', 'vehicle_type_code2':'vehicle_type_code_2'}, inplace=True)

In [316]:
import requests
import pandas as pd

# Fetch data from the API
additional_columns = requests.get("https://data.cityofnewyork.us/resource/h9gi-nx95.json")
additional_columns_data = additional_columns.json()
additional_df = pd.DataFrame(additional_columns_data)
additional_df.set_index('collision_id', inplace=True)

In [317]:
df.index = df.index.astype(str)
additional_df.index = additional_df.index.astype(str)

In [318]:
# rename columns contributing_factor_vehicle to contributing_factor_vehicle_1 
df.rename(columns={'contributing_factor_vehicle':'contributing_factor_vehicle_1'}, inplace=True)

In [319]:
df['contributing_factor_vehicle_2'] = additional_df['contributing_factor_vehicle_2']
df['contributing_factor_vehicle_3'] = additional_df['contributing_factor_vehicle_3']
df['contributing_factor_vehicle_4'] = additional_df['contributing_factor_vehicle_4']
df['contributing_factor_vehicle_5'] = additional_df['contributing_factor_vehicle_5']

In [320]:
df['vehicle_type_code_1'] = additional_df['vehicle_type_code1']
df['vehicle_type_code_2'] = additional_df['vehicle_type_code2']
df['vehicle_type_code_3'] = additional_df['vehicle_type_code_3']
df['vehicle_type_code_4'] = additional_df['vehicle_type_code_4']
df['vehicle_type_code_5'] = additional_df['vehicle_type_code_5']

In [321]:
crash_datetime = df['crash_datetime']
df.drop(labels=['crash_datetime'], axis=1, inplace=True)
df.insert(0, 'crash_datetime', crash_datetime)

In [322]:
df

,crash_datetime,crash_day_of_week,street_address,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,...,borough,zip_code,latitude,longitude,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
collision_id,,,,,,,,,,,,,,,,,,,,,
4513547,2022-03-26 11:45:00,5,Queensboro Bridge Upper,1,0,0,0,0,0,1,...,NaN,10044,40.756929,-73.954755,NaN,NaN,NaN,NaN,NaN,NaN
4541903,2022-06-29 06:55:00,2,Throgs Neck Bridge,0,0,0,0,0,0,0,...,NaN,10465,40.805080,-73.793780,NaN,NaN,NaN,NaN,NaN,NaN
4542336,2022-06-29 16:00:00,2,Williamsburg Bridge Outer Roadwa,1,0,0,0,0,0,1,...,NaN,11211,40.713595,-73.971962,NaN,NaN,NaN,NaN,NaN,NaN
4545699,2022-07-12 17:50:00,1,44 Empire Boulevard,0,0,0,0,0,0,0,...,Brooklyn,11225,40.663303,-73.960490,NaN,NaN,NaN,NaN,NaN,NaN
4512922,2022-03-23 10:00:00,2,71 East Drive,0,0,0,0,0,0,0,...,NaN,11225,40.661927,-73.964893,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4647581,2022-09-15 19:49:00,3,East New York Avenue & Rockaway Parkway,0,0,0,0,0,0,0,...,Brooklyn,11213,40.665916,-73.925470,NaN,NaN,NaN,NaN,NaN,NaN
4647839,2022-07-21 22:05:00,3,Cross Island Parkway,0,0,0,0,0,0,0,...,NaN,11411,40.697162,-73.727529,NaN,NaN,NaN,NaN,NaN,NaN
4647650,2022-12-31 02:00:00,5,Tilden Avenue,1,0,0,0,0,0,1,...,NaN,11203,40.647121,-73.943381,NaN,NaN,NaN,NaN,NaN,NaN


In [323]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.usa.com/rank/new-york-state--population-density--zip-code-rank.htm#google_vignette'

response = requests.get(url)
page_text = response.text

soup = BeautifulSoup(page_text)

table_rows = soup.find_all('tr')

data = []
for row in table_rows[2:]:  # Skipping the first two rows
    cols = row.find_all('td')
    if len(cols) == 3:  # Ensuring the row has the correct number of columns
        rank, density, zip_population = cols
        density_value = density.text.split('/')[0]  # Extracting just the density value
        zipcode, population = zip_population.text.split(' / ')
        data.append([zipcode, density_value, population])
demo_df = pd.DataFrame(data, columns=['Zipcode', 'Density', 'Population'])
demo_df = demo_df.rename(columns={"Zipcode": "zip_code"})

In [324]:
df = df.reset_index().merge(demo_df, on='zip_code', how='left').set_index('collision_id')


In [325]:
df['Population'] = df['Population'].str.replace(',', '').astype(float)

In [326]:
collision_by_zip_df = df.groupby('zip_code').size()
collision_by_zip_df = collision_by_zip_df.reset_index().rename(columns={"zip_code":"zipcodes",0:"# of Collisions"})
collision_by_zip_df['# of Collisions'] = collision_by_zip_df['# of Collisions'].astype('int')
collision_by_zip_df['zipcodes'] = collision_by_zip_df['zipcodes'].astype('str')
collision_by_zip_df.set_index('zipcodes', inplace=True)
collision_by_zip_df.head()

,# of Collisions
zipcodes,
10000,30
10001,590
10002,880
10003,415
10004,89


In [327]:
engine = create_engine('sqlite:///../Resources/collision_db.sqlite')
metadata = MetaData()

In [328]:
collision_by_zip = Table('collision_by_zip', metadata,
    Column('zipcodes', String, primary_key=True),
    Column('# of Collisions', Integer),
)
metadata.create_all(engine)

In [329]:
collision_by_zip_df.to_sql('collision_by_zip', engine, if_exists='append', index=True)

245

In [330]:
## add an attribute that classifies the hour of the day
df["crash_hour"] = df['crash_datetime'].dt.hour

def classify_hour(hour):
    if 4 <= hour < 7:
        return 'Early Morning'
    elif 7 <= hour < 10 or 15 <= hour < 20:
        return 'Rush Hours'
    elif 22 <= hour or hour < 4:
        return 'Late Night'
    else:
        return 'Other Hours'

df['crash_hour_category'] = df['crash_hour'].apply(classify_hour)    
df = df.drop('crash_hour', axis=1)
column_to_move = df.pop('crash_hour_category')
df.insert(2, 'crash_hour_category', column_to_move)

In [331]:
## check for missing values:
# drop attributes of contributing factors that only have value "unspecified":
contributing_factor_columns = [
    'contributing_factor_vehicle_1',
    'contributing_factor_vehicle_2',
    'contributing_factor_vehicle_3',
    'contributing_factor_vehicle_4',
    'contributing_factor_vehicle_5'
]

# Check if all values in each contributing factor column are "Unspecified"
columns_to_drop = [col for col in contributing_factor_columns if all(df[col].dropna() == 'Unspecified')]

# Drop the columns
df.drop(columns=columns_to_drop, inplace=True)

#drop attributes of which all the values are missing:
df.dropna(axis=1, how='all',inplace = True)

In [332]:
# combine the contributing factors into one attribute:
df["contributing_factor"] = df["contributing_factor_vehicle_1"]
df.drop(["contributing_factor_vehicle_1","contributing_factor_vehicle_2"],axis = 1,inplace= True)

In [333]:
df['zip_code'] = df['zip_code'].astype('str')
df[df['zip_code'].str.len() != 5]['zip_code']

collision_id
4524418         7514
4526434         7307
4527885         7006
4528539         7047
4562093    113541906
4585574         8721
4591216         7305
4591962         8721
Name: zip_code, dtype: object

In [334]:
df.loc[df['zip_code'] == '113541906','zip_code'] = "11354"

In [335]:
## drop the rows that have four-digit zip code
df.drop(df[df['zip_code'].str.len() != 5].index, inplace = True)

In [336]:
df["number_of_injuries"] = df["number_of_cyclist_injured"] + df["number_of_motorist_injured"] + df["number_of_pedestrians_injured"] + df["number_of_persons_injured"]
df["number_of_deaths"] = df["number_of_cyclist_killed"] + df["number_of_motorist_killed"] + df["number_of_pedestrians_killed"] + df["number_of_persons_killed"]

In [114]:
df["number_of_deaths"].value_counts()

number_of_deaths
0    103506
2       248
1        20
4         6
6         2
Name: count, dtype: int64

In [115]:
# redefine the severity of accidents
def categorize_severity(row):
    if row['number_of_deaths'] >= 3:
        return 'Very Fatal'
    elif row['number_of_deaths'] > 0:
        return 'Fatal'
    elif row['number_of_injuries'] >= 10:
        return 'Serious (High)'
    elif row['number_of_injuries'] >= 3:
        return 'Serious (Medium)'
    elif row['number_of_injuries'] > 0:
        return 'Serious (Low)'
    else:
        return 'Minor'

# Apply the function to create a new column 'severity_of_accident'
df['severity_of_accident'] = df.apply(categorize_severity, axis=1)

# Display the resulting DataFrame
print(df[['number_of_injuries', 'number_of_deaths', 'severity_of_accident']])

              number_of_injuries  number_of_deaths severity_of_accident
collision_id                                                           
4513547                        2                 0        Serious (Low)
4541903                        0                 0                Minor
4542336                        2                 0        Serious (Low)
4545699                        0                 0                Minor
4512922                        0                 0                Minor
...                          ...               ...                  ...
4647581                        0                 0                Minor
4647839                        0                 0                Minor
4647650                        2                 0        Serious (Low)
4648107                        0                 0                Minor
4648224                        0                 0                Minor

[103782 rows x 3 columns]


In [116]:
# replace null value in vehicle_type_code_1 with 'Unspecified'
df['vehicle_type_code_1'].fillna('Unspecified', inplace=True)

In [117]:
statistics_df = pd.DataFrame({
    "number_of_injuries": [df["number_of_injuries"].sum()],  
    "number_of_deaths": [df["number_of_deaths"].sum()],      
    "number_of_accidents": [df.shape[0]]                     
})
statistics_df.head()

,number_of_injuries,number_of_deaths,number_of_accidents
0,101394,552,103782


In [118]:
motor_statistic = Table('motor_statistic', metadata,
                        Column('index', Integer, primary_key=True),
                        Column('number_of_injuries', Integer),
                        Column('number_of_deaths', Integer),
                        Column('number_of_accidents', Integer),
)
metadata.create_all(engine)

InvalidRequestError: Table 'motor_statistic' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
statistics_df.to_sql('motor_statistic', engine, if_exists='append', index=False)

1

In [119]:
import json

# Define the file path
file_path = '../Resources/nyc_geojson_by_zip.json'

# Open the file and load the GeoJSON data
with open(file_path, 'r') as file:
    geojson_data = json.load(file)

In [120]:
# add the collision count to the geojson data
for feature in geojson_data['features']:
    zipcode = feature['properties']['postalCode']
    if zipcode in collision_by_zip_df.index:
        feature['properties']['collision_count'] = collision_by_zip_df.loc[zipcode]['# of Collisions']
    else:
        feature['properties']['collision_count'] = 0

In [121]:
import numpy as np
# Custom function to handle non-serializable data types
def convert(o):
    if isinstance(o, np.integer):
        return int(o)
    raise TypeError

# Use json.dump with the custom function to handle non-serializable data
with open('../static/data/nyc_geojson_by_zip_with_counts.json', 'w') as file:
    json.dump(geojson_data, file, default=convert)

In [122]:
df.drop(['borough'], axis = 1, inplace = True)

KeyError: "['borough'] not found in axis"

In [123]:
df.to_csv('../Resources/collision_2022_classified.csv', index=False)

In [124]:
motor_collisions = Table('motor_collisions', metadata,
                         Column('collision_id', Integer, primary_key=True),
                         Column('crash_datetime', DateTime),
                         Column('crash_day_of_week', Integer),
                         Column('crash_hour_category', String),
                         Column('street_address', String), 
                         Column('zip_code', String),
                         Column('latitude', Float),
                         Column('longitude', Float),
                         Column('number_of_persons_injured', Integer),
                         Column('number_of_persons_killed', Integer),
                         Column('number_of_pedestrians_injured', Integer),
                         Column('number_of_pedestrians_killed', Integer),
                         Column('number_of_cyclist_injured', Integer),
                         Column('number_of_cyclist_killed', Integer),
                         Column('number_of_motorist_injured', Integer),
                         Column('number_of_motorist_killed', Integer),
                         Column('contributing_factor', String),
                         Column('number_of_injuries', Integer),
                         Column('number_of_deaths', Integer),
                         Column('vehicle_type_code_1', String),
                         Column('vehicle_type_code_2', String),
                         Column('vehicle_type_code_3', String),
                         Column('vehicle_type_code_4', String),
                         Column('severity_of_accident', String),
                         Column('Population', Float),
                         Column('Density', Float)
                         )
metadata.create_all(engine)

InvalidRequestError: Table 'motor_collisions' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
df.to_sql('motor_collisions', con=engine, if_exists='append', index=True)

103782

In [ ]:
weather_df = pd.read_csv("../Resources/weather_2022.csv")
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

columns_to_drop = ['precipprob','name','feelslikemax','feelslikemin','feelslike','precipcover','stations','description','conditions','moonphase','sealevelpressure','cloudcover','snowdepth','winddir','solarradiation','solarenergy','dew','uvindex','windgust']
weather_df = weather_df.drop(columns_to_drop, axis = 1)

weather_df['sunrise'] = pd.to_datetime(weather_df['sunrise']).dt.strftime('%H:%M')
weather_df['sunset'] = pd.to_datetime(weather_df['sunset']).dt.strftime('%H:%M')
weather_df['severerisk'] = weather_df['severerisk'].fillna(0)
weather_df['preciptype'] = weather_df['preciptype'].fillna('dry')
weather_df['preciptype'] = np.where(weather_df['preciptype'].str.contains('rain', case=True),1,0) # 1 represents rain, 0 represents dry
weather_df['preciptype'] = np.where((weather_df['precip'] == 0) & (weather_df['preciptype'] != 'dry'),0,weather_df['preciptype'])
weather_df['snow'] = np.where(weather_df['snow']>0,1,0) # 1 represents snow, 0 represents no snow
weather_df['icon'] = np.where((weather_df['icon'] == 'partly-cloudy-day'),'cloudy',weather_df['icon'])

#weather_df.groupby(weather_df['temp']).size()

temperature_bins = [-10, 20, 40, 60, 80, 90, 100]
temperature_labels = ['Freezing Cold', 'Cold', 'Moderate', 'Warm', 'Hot','Unbearable Hot']
precip_bins = [-0.01,0.0001,0.1,0.3,float('inf')]
precip_labels = ['Dry', 'Light', 'Moderate', 'Heavy']
humid_bins = [-float('inf'),50,80,float('inf')]
humid_labels = ['Low','High','Very High']
wind_bins = [4,7.1,12.1,18.1,24.1,31.1]
wind_labels = ['Flags ripple','Flags wave','Leaves scatter','Small trees sway','Umbrellas not working']
vis_bins = [3,6,9,float('inf')]
vis_labels = ["Poor","Moderate","Good"]

weather_df['temperature_category'] = pd.cut(weather_df['temp'], bins=temperature_bins, labels=temperature_labels)
weather_df['precip_category'] = pd.cut(weather_df['precip'], bins=precip_bins, labels=precip_labels)
weather_df['humid_category'] = pd.cut(weather_df['humidity'], bins=humid_bins, labels=humid_labels)
weather_df['wind_level'] = pd.cut(weather_df['windspeed'], bins=wind_bins, labels=wind_labels)
weather_df['vis_level'] = pd.cut(weather_df['visibility'], bins=vis_bins, labels=vis_labels)

weather_df['temperature_category'] = weather_df['temperature_category'].astype('string')
weather_df['precip_category'] = weather_df['precip_category'].astype('string')
weather_df['humid_category'] = weather_df['humid_category'].astype('string')
weather_df['wind_level'] = weather_df['wind_level'].astype('string')
weather_df['vis_level'] = weather_df['vis_level'].astype('string')

In [ ]:
weather_info = Table('weather_2022_info', metadata,
                         Column('datetime', DateTime, primary_key=True),
                         Column('tempmax', Float),
                         Column('tempmin', Float),
                         Column('temp', Float), 
                         Column('humidity', Float),
                         Column('precip', Float),
                         Column('preciptype', Integer),
                         Column('snow', Integer),
                         Column('windspeed', Float),
                         Column('visibility', Float),
                         Column('severerisk', Float),
                         Column('sunrise', String),
                         Column('sunset', String),
                         Column('icon', String),
                         Column('temperature_category', String),
                         Column('precip_category', String),
                         Column('number_of_motorist_killed', Integer),
                         Column('humid_category', String),
                         Column('wind_level', String),
                         Column('vis_level', String)
                         )
metadata.create_all(engine)

In [ ]:
weather_df.to_sql('weather_2022_info', con=engine, if_exists='append',index=False)

365

In [125]:
df['date'] = df['crash_datetime'].apply(lambda x : x.strftime('%Y-%m-%d'))
df['date'] = pd.to_datetime(df['date'])
df['hour'] = df['crash_datetime'].dt.hour

In [ ]:
## create a new index so that we can include the everyday weather info for each zip code 
weather_df = weather_df.rename(columns={'datetime': 'date'})
weather_df.set_index("date",inplace = True)

In [128]:
allzipcodes_weather_df = pd.merge(df, weather_df, on='date', how='left').set_index(df.index)

In [129]:
allzipcodes_weather_df['zip_code'] = allzipcodes_weather_df['zip_code'].astype('str')
allzipcodes_weather_df['Density'] = allzipcodes_weather_df['Density'].str.replace(',', '').astype(float)
allzipcodes_weather_df['Density_rank'] = allzipcodes_weather_df['Density'].rank(ascending=False, method='dense')
zip_code_counts = allzipcodes_weather_df['zip_code'].value_counts()
allzipcodes_weather_df['zip_code_count'] = allzipcodes_weather_df['zip_code'].map(zip_code_counts)
allzipcodes_weather_df['total_inflicted'] = allzipcodes_weather_df['number_of_deaths'] + allzipcodes_weather_df['number_of_injuries']

In [130]:
allzipcodes_weather_df

,crash_datetime,crash_day_of_week,crash_hour_category,street_address,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,sunset,icon,temperature_category,precip_category,humid_category,wind_level,vis_level,Density_rank,zip_code_count,total_inflicted
collision_id,,,,,,,,,,,,,,,,,,,,,
4513547,2022-03-26 11:45:00,5,Other Hours,Queensboro Bridge Upper,1,0,0,0,0,0,...,19:14,rain,Moderate,Light,High,Leaves scatter,Good,65.0,117,2
4541903,2022-06-29 06:55:00,2,Early Morning,Throgs Neck Bridge,0,0,0,0,0,0,...,20:31,rain,Warm,Light,High,Leaves scatter,Good,154.0,643,0
4542336,2022-06-29 16:00:00,2,Rush Hours,Williamsburg Bridge Outer Roadwa,1,0,0,0,0,0,...,20:31,rain,Warm,Light,High,Leaves scatter,Good,83.0,1108,2
4545699,2022-07-12 17:50:00,1,Rush Hours,44 Empire Boulevard,0,0,0,0,0,0,...,20:27,cloudy,Warm,Dry,High,Leaves scatter,Good,37.0,608,0
4512922,2022-03-23 10:00:00,2,Other Hours,71 East Drive,0,0,0,0,0,0,...,19:11,rain,Moderate,Light,Low,Leaves scatter,Good,37.0,608,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4647581,2022-09-15 19:49:00,3,Rush Hours,East New York Avenue & Rockaway Parkway,0,0,0,0,0,0,...,19:05,clear-day,Warm,Dry,Low,Small trees sway,Good,48.0,690,0
4647839,2022-07-21 22:05:00,3,Late Night,Cross Island Parkway,0,0,0,0,0,0,...,20:21,rain,Hot,Moderate,High,Leaves scatter,Good,140.0,818,0
4647650,2022-12-31 02:00:00,5,Late Night,Tilden Avenue,1,0,0,0,0,0,...,16:38,rain,Moderate,Moderate,Very High,Flags ripple,Poor,93.0,1539,2


In [131]:
allzipcodes_weather_df.to_csv('../Resources/allzipcodes_collision_weather_df.csv', index=True)

In [ ]:
allzipcodes_weather_df.columns

Index(['crash_datetime', 'crash_day_of_week', 'crash_hour_category',
       'street_address', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'vehicle_type_code_1',
       'vehicle_type_code_2', 'zip_code', 'latitude', 'longitude',
       'vehicle_type_code_3', 'vehicle_type_code_4', 'Density', 'Population',
       'contributing_factor', 'number_of_injuries', 'number_of_deaths',
       'severity_of_accident', 'date', 'hour', 'tempmax', 'tempmin', 'temp',
       'humidity', 'precip', 'preciptype', 'snow', 'windspeed', 'visibility',
       'severerisk', 'sunrise', 'sunset', 'icon', 'temperature_category',
       'precip_category', 'humid_category', 'wind_level', 'vis_level',
       'Density_rank', 'zip_code_count', 'total_inflicted'],
      dtype='object')

In [ ]:
weather_collision = Table('weather_collision', metadata,
    Column('row_id', String, primary_key= True),
    Column('crash_datetime', String),  
    Column('crash_day_of_week', Integer),
    Column('crash_hour_category', String),
    Column('street_address', String),
    Column('number_of_persons_injured', Integer),
    Column('number_of_persons_killed', Integer),
    Column('number_of_pedestrians_injured', Integer),
    Column('number_of_pedestrians_killed', Integer),
    Column('number_of_cyclist_injured', Integer),
    Column('number_of_cyclist_killed', Integer),
    Column('number_of_motorist_injured', Integer),
    Column('number_of_motorist_killed', Integer),
    Column('vehicle_type_code_1', String),
    Column('vehicle_type_code_2', String),
    Column('zip_code', String), 
    Column('latitude', Float),
    Column('longitude', Float),
    Column('vehicle_type_code_3', String),
    Column('vehicle_type_code_4', String),
    Column('Density', Float),  
    Column('Population', Float),
    Column('contributing_factor', String),
    Column('number_of_injuries', Integer),
    Column('number_of_deaths', Integer),
    Column('severity_of_accident', String),
    Column('hour', Integer),
    Column('date', String),
    Column('tempmax', Float),
    Column('tempmin', Float),
    Column('temp', Float),
    Column('humidity', Float),
    Column('precip', Float),
    Column('preciptype', Integer),
    Column('snow', Integer),
    Column('windspeed', Float),
    Column('visibility', Float),
    Column('severerisk', Float),
    Column('sunrise', String),
    Column('sunset', String),
    Column('icon', String),
    Column('temperature_category', String),
    Column('precip_category', String),
    Column('humid_category', String),
    Column('wind_level', String),
    Column('vis_level',String),
    Column('Density_rank',Integer),
    Column('zip_code_count',Integer),
    Column('total_inflicted', Integer)
)

metadata.create_all(engine)

In [ ]:
allzipcodes_weather_df.to_sql('weather_collision', con=engine, if_exists='append',index=False)

103782

In [ ]:
from sqlalchemy.ext.automap import automap_base
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

/var/folders/_7/4zts0j8d4413ps8jq10scqsr0000gn/T/ipykernel_97469/70205817.py:3: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


['motor_statistic']